In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import math
import zipfile
import json
import collections
import logging
import time
import datetime
import random
from sklearn.metrics import classification_report

In [7]:
logging.basicConfig(level=logging.DEBUG)

def __preprocess_dataframe(df, features, metadata=None):
    """
    If metadata is passed, return also the labels
    """
    # Trim columns name, replace whitespaces from columns name
    df = df.rename(columns=lambda x: x.strip().replace(" ", "_"))
    # Keep only features and label
    df = df[features + ["Label"]]
    if metadata is None:
        # Remove label column from features
        df.pop('Label')
    else:
        # Labels between training and set dataset may be different, map them using metadata
        labels = df['Label'].unique()
        for l in labels:
            if l in metadata['Label']['01-12->03-11']:
                df['Label'].replace(to_replace=l, value=metadata['Label']['01-12->03-11'][l], inplace=True)
    return df


def __get_features(archive, metadata):
    """
    Retrieve the features from the archive
    """
    features = []
    # Get a flat list of all best features
    for dataset_label in metadata['Label']['Features']:
        features += metadata['Label']['Features'][dataset_label]
    features = list(set(features))
    features = [fc.replace(" ", "_") for fc in features]
    # Use the first csv file in the archive to extract column features
    file = next((file for file in archive.namelist() if file.endswith(".csv")), None)
    df = __preprocess_dataframe(pd.read_csv(archive.open(file), dtype={85: str}), features)
    feature_columns = []
    for key in df.keys():
        feature_columns.append(tf.feature_column.numeric_column(key=key))
    return feature_columns

def load_dataset(zipfile_path, metadata_path, random_state=None, csvs=None, chunksize=None):

    logging.debug('Loading metadata...')
    with open(metadata_path) as metadata_file:
        metadata = json.load(metadata_file)

    logging.debug('Opening archive...')
    archive = zipfile.ZipFile(zipfile_path, 'r')

    feature_columns = __get_features(archive, metadata)

    sets = []
    for file in archive.namelist():
        # Load all the files if csvs parameter is None
        # otherwise check that the file is listed in csvs
        if csvs is None:
            load_file = file.endswith(".csv")
        else:
            load_file = any(file.endswith(t) for t in csvs)
        if load_file:
            logging.debug('     > Loading %s...', file)
            # Load the entire csv file only if chunksize is not specified
            if chunksize is None:
                df = __preprocess_dataframe(
                    df = pd.read_csv(
                        archive.open(file),
                        dtype={85: str}
                    ),
                    features = [fc.key.replace(" ", "_") for fc in feature_columns],
                    metadata = metadata
                )
                sets.append(df)
            else:
                for chunk in pd.read_csv(archive.open(file), dtype={85: str}, chunksize=chunksize):
                    df = __preprocess_dataframe(
                        df = chunk,
                        features = [fc.key.replace(" ", "_") for fc in feature_columns],
                        metadata = metadata
                    )
                    sets.append(df)
    # Merge the dataframes into a single one and shuffle it, random_state assures reproducibility
    logging.debug('Merging and shuffling...')
    df = pd.concat(sets, ignore_index=True)
    if random_state is None:
        return df
    else:
        return df.sample(frac=1, random_state=random_state)

In [3]:
logging.debug(datetime.datetime.now())

DEBUG:root:2020-08-08 07:02:54.842309


# Preprocessing

In [4]:
start = time.time()
# Preprocess the dataset
train_df = load_dataset(
    zipfile_path = "./data/CSV-01-12.zip",
    metadata_path = "./data/metadata.json",
    random_state = 1,
    csvs = ['UDPLag.csv', 'Syn.csv', 'DrDoS_UDP.csv', 'DrDoS_NetBIOS.csv', 'DrDoS_MSSQL.csv', 'DrDoS_LDAP.csv'],
    chunksize=9**6
)
end = time.time()
preprocess_time = (end - start)
logging.debug('Preprocessing time: {t}'.format(t=datetime.timedelta(seconds=preprocess_time)))

DEBUG:root:Loading metadata...
DEBUG:root:Opening archive...
DEBUG:root:     > Loading 01-12/UDPLag.csv...
DEBUG:root:     > Loading 01-12/Syn.csv...
DEBUG:root:     > Loading 01-12/DrDoS_UDP.csv...
DEBUG:root:     > Loading 01-12/DrDoS_NetBIOS.csv...
DEBUG:root:     > Loading 01-12/DrDoS_MSSQL.csv...
DEBUG:root:     > Loading 01-12/DrDoS_LDAP.csv...
DEBUG:root:Merging and shuffling...
DEBUG:root:Preprocessing time: 0:03:36.292166


In [5]:
feature_columns = [tf.feature_column.numeric_column(key=key) for key in train_df.keys() if key != "Label" ]
labels = ["BENIGN", "Syn", "UDPLag", "UDP", "LDAP", "MSSQL", "NetBIOS", "WebDDoS"]

In [6]:
chunk_size = 9**6
train_dfs = np.split(train_df, range(chunk_size, math.ceil(train_df.shape[0] / chunk_size) * chunk_size, chunk_size))
del train_df
round = len(train_dfs)

# Training

In [7]:
def input_fn(df, training, batch_size=32):
    '''
    An input function for training or evaluating
    '''
    # Convert the inputs to a Dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(df[[ x for x in list(df.columns.values) if x != "Label" ]]), df["Label"]))
    # Shuffle and repeat if you are in training mode
    if training:
        dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)

In [8]:
def run_config(hparams, model_dir):
    classifier = tf.estimator.DNNClassifier(
        hidden_units = hparams['HIDDEN_UNITS'],
        feature_columns = feature_columns,
        n_classes = len(labels),
        label_vocabulary = labels,
        batch_norm = True,
        model_dir=model_dir,
        dropout = hparams['DROPOUT'],
        optimizer = lambda: tf.keras.optimizers.Adam(
          learning_rate = tf.compat.v1.train.exponential_decay(
            learning_rate = hparams['LEARNING_RATE'],
            global_step = tf.compat.v1.train.get_global_step(),
            decay_steps = 10000,
            decay_rate = 0.96)
        ),
        config = tf.estimator.RunConfig(
          tf_random_seed = 9,
          log_step_count_steps = 10 ** 5
        )
      )
    
    train_df_count = int(round*(80/100))
    # Train over #train_df_count dataframes
    for train_df in train_dfs[:train_df_count]:
        classifier.train(input_fn=lambda: input_fn(train_df, training=True), steps=10**4)
    accuracies = [] 
    # Evaluate over the remaining dataframes
    for evaluate_df in train_dfs[train_df_count:]:
        accuracies.append(classifier.evaluate(input_fn=lambda: input_fn(evaluate_df, training=False))['accuracy'])
    return {
        "hparams": hparams,
        "classifier": classifier,
        "accuracy": sum(accuracies)/len(accuracies)
    }

In [9]:
start = time.time()

session_num = 0
session_runs = []
best_run = None

HIDDEN_UNITS = [[60, 30, 20], [60, 40, 30, 20]]
DROPOUT = [0.1, 0.2]
LEARNING_RATE = [0.1, 0.3]
for hidden_units in HIDDEN_UNITS:
    for dropout in DROPOUT:
        for learning_rate in LEARNING_RATE:
            hparams = {}
            hparams['HIDDEN_UNITS'] = hidden_units
            hparams['DROPOUT'] = dropout
            hparams['LEARNING_RATE'] = learning_rate
            logging.debug("Session #%d" % session_num)
            logging.debug('hparams: %s', hparams)
            logging.debug(datetime.datetime.now().time())
            run = run_config(hparams,"./model/"+str(session_num))
            session_runs.append(run)
            if best_run is None or best_run["accuracy"] < run["accuracy"]:
                # Set current model as the classifier to export
                best_run = run
            session_num += 1
end = time.time()
train_time = (end - start)

DEBUG:root:Session #0
DEBUG:root:hparams: {'HIDDEN_UNITS': [60, 30, 20], 'DROPOUT': 0.1, 'LEARNING_RATE': 0.1}
DEBUG:root:07:06:37.878794


INFO:tensorflow:Using config: {'_model_dir': './model/0', '_tf_random_seed': 9, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100000, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': './model/0', '_tf_random_seed': 9, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100000, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


...

INFO:tensorflow:Finished evaluation at 2020-08-08-11:49:03


INFO:tensorflow:Saving dict for global step 240000: accuracy = 0.7579784, average_loss = 2.2616086, global_step = 240000, loss = 2.2615995


INFO:tensorflow:Saving dict for global step 240000: accuracy = 0.7579784, average_loss = 2.2616086, global_step = 240000, loss = 2.2615995


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 240000: ./model/7/model.ckpt-240000


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 240000: ./model/7/model.ckpt-240000


In [10]:
logging.debug('Training time: {t}'.format(t=datetime.timedelta(seconds=train_time)))

DEBUG:root:Training time: 4:42:25.478339


In [17]:
best_run

{'hparams': {'HIDDEN_UNITS': [60, 40, 30, 20],
  'DROPOUT': 0.1,
  'LEARNING_RATE': 0.3},
 'classifier': <tensorflow_estimator.python.estimator.canned.dnn.DNNClassifierV2 at 0x7fdf853c2780>,
 'accuracy': 0.849530816078186}

In [11]:
# Save the model
logging.debug("Saving model...")
classifier = best_run["classifier"]
serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(tf.feature_column.make_parse_example_spec(feature_columns))
estimator_path = classifier.export_saved_model("./model", serving_input_receiver_fn=serving_input_fn)
logging.debug("- Estimator path")
logging.debug(estimator_path)

DEBUG:root:Saving model...


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Assets added to graph.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: ./model/temp-b'1596887343'/saved_model.pb


INFO:tensorflow:SavedModel written to: ./model/temp-b'1596887343'/saved_model.pb
DEBUG:root:- Estimator path
DEBUG:root:b'./model/1596887343'


# Preprocessing

In [9]:
start = time.time()
# Preprocess the dataset
df = load_dataset(
    zipfile_path = "./data/CSV-03-11.zip",
    metadata_path = "./data/metadata.json",
    csvs = ['Syn.csv', 'UDPLag.csv', 'UDP.csv', 'LDAP.csv', 'MSSQL.csv', 'NetBIOS.csv'],
    chunksize=8**6
)
end = time.time()
preprocess_time = (end - start)
logging.debug('Preprocessing time: {t}'.format(t=datetime.timedelta(seconds=preprocess_time)))

DEBUG:root:Loading metadata...
DEBUG:root:Opening archive...
DEBUG:root:     > Loading 03-11/UDPLag.csv...
DEBUG:root:     > Loading 03-11/UDP.csv...
DEBUG:root:     > Loading 03-11/LDAP.csv...
DEBUG:root:     > Loading 03-11/MSSQL.csv...
DEBUG:root:     > Loading 03-11/NetBIOS.csv...
DEBUG:root:     > Loading 03-11/Syn.csv...
DEBUG:root:Merging and shuffling...
DEBUG:root:Preprocessing time: 0:04:07.754210


In [11]:
# Load the DNN models
model = tf.saved_model.load(estimator_path)

# Test

In [13]:
def predict(model, df):
    """
    returns the predicted label given a dataframe of features
    """
    feature = {k: tf.train.Feature(float_list=tf.train.FloatList(value=[v])) for k, v in dict(df).items()}
    example = tf.train.Example(features=tf.train.Features(feature=feature))
    prediction = model.signatures["predict"](examples=tf.constant([example.SerializeToString()]))
    return prediction["classes"].numpy()[0][0].decode("utf-8")

start = time.time()

features_name = [feature for feature in list(df) if feature != "Label"]
# Test the model
logging.debug("Testing model...")
# Set random seed for reproducibility
random.seed(9)

y_test = []
y_pred = []

for i in random.sample(range(0, len(df)), k=10**3):
    y_test.append(df['Label'].iloc[i])
    y_pred.append(predict(model, df[features_name].iloc[i]))

metrics = classification_report(y_test, y_pred)

logging.debug("Metrics")
logging.debug(metrics)

end = time.time()
test_time = (end - start)
logging.debug('Test time: {t}'.format(t=datetime.timedelta(seconds=test_time)))


DEBUG:root:Testing model...
/home/jovyan/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
DEBUG:root:Metrics
DEBUG:root:              precision    recall  f1-score   support

      BENIGN       0.00      0.00      0.00         1
        LDAP       0.93      0.96      0.94        96
       MSSQL       0.93      0.66      0.77       270
     NetBIOS       1.00      0.98      0.99       183
         Syn       1.00      0.98      

In [ ]:
logging.debug(datetime.datetime.now())